In [ ]:
# Instalación PySpark
!pip install -q PySpark



In [1]:
# Creamos la sesión de Spark con soporte para Structured Streaming

from pyspark.sql import SparkSession

spark= (
    SparkSession.builder.
      appName("ProyectoMetroSantiago").
      config("spark.executor.memory","1g").
      config("spark.driver.memory","1g").
      config("spark.sql.shuffle.partitions","2").
      getOrCreate()
)

In [2]:
print(f"Spark version: {spark.version}")

Spark version: 3.5.1


In [3]:
# Importar 2 librerias necesarias para simular la lectura de streams
import shutil
import os

# Shutil permitir copiar, mover y eliminar archivos y carpetas
# OS nos permite interactuar con el Sist. Operativo para crear directorios

# Crear una carpeta para lamacenar los minilotes (microbatch)
os.makedirs("stream_data", exist_ok=True)

# Definir nuestra URL de la data
dataset = "metro_santiago.csv"

for i in range (1,4):
  shutil.copy(dataset, f"stream_data/lote{i}.csv")

In [4]:
# Definir el esquema del DF porque no se puede inferir en Streaming
from pyspark.sql.types import StructType, StringType, IntegerType

# Acá defino la estructura que va a tomar el df, Streaming no lo infiere
schema = (
    StructType().
    add("id_evento", IntegerType()).
    add("linea", StringType()).
    add("estacion", StringType()).
    add("tipo_incidente", StringType()).
    add("duracion_minutos", IntegerType()).
    add("hora", StringType())
)

# Creamos el df con la función readStream()
df_stream = (
    spark.readStream.
    schema(schema).
    option("sep",",").
    option("header","true").
    option("maxFilesPerTrigger", 1).
    csv("stream_data").
    select("estacion","linea","tipo_incidente","duracion_minutos")
)

In [5]:
df_prueba = spark.read.schema(schema).option("header","true").csv("stream_data")

df_prueba.printSchema()
df_prueba.show(2)

root
 |-- id_evento: integer (nullable = true)
 |-- linea: string (nullable = true)
 |-- estacion: string (nullable = true)
 |-- tipo_incidente: string (nullable = true)
 |-- duracion_minutos: integer (nullable = true)
 |-- hora: string (nullable = true)

+---------+-----+-----------+---------------+----------------+-----+
|id_evento|linea|   estacion| tipo_incidente|duracion_minutos| hora|
+---------+-----+-----------+---------------+----------------+-----+
|        1|   L2|   Lo Prado|Falla eléctrica|              15|11:45|
|        2|   L5|Irarrázaval|     Evacuación|              15|20:00|
+---------+-----+-----------+---------------+----------------+-----+
only showing top 2 rows



In [6]:
# Transformar el DF
from pyspark.sql.functions import count

df_agrupado = df_stream.groupBy("tipo_incidente").agg(count("*").alias("total"))

spark.conf.set("spark.sql.shuffle.partitions","1")


In [8]:
# Mostrar la info. Con formato Streaming

import shutil
shutil.rmtree("/tmp/chk_metro_fb", ignore_errors=True)

def imprimir_batch(df, epoch_id):
  df.orderBy("tipo_incidente").show(n=50, truncate=False)

query = (
    df_agrupado.writeStream
    .foreachBatch(imprimir_batch)
    .outputMode("complete")
    .option("checkpointLocation","/tmp/chk_metro_fb")
    .trigger(processingTime="2 seconds")
    .start()
)

query.awaitTermination(10)
query.stop()

+------------------------+-----+
|tipo_incidente          |total|
+------------------------+-----+
|Evacuación              |8    |
|Falla eléctrica         |14   |
|Interrupción de servicio|11   |
|Mantenimiento           |12   |
|Retraso                 |5    |
+------------------------+-----+

+------------------------+-----+
|tipo_incidente          |total|
+------------------------+-----+
|Evacuación              |16   |
|Falla eléctrica         |28   |
|Interrupción de servicio|22   |
|Mantenimiento           |24   |
|Retraso                 |10   |
+------------------------+-----+

+------------------------+-----+
|tipo_incidente          |total|
+------------------------+-----+
|Evacuación              |24   |
|Falla eléctrica         |42   |
|Interrupción de servicio|33   |
|Mantenimiento           |36   |
|Retraso                 |15   |
+------------------------+-----+

